In [ ]:
import os from pathlib 
import Path
import pandas as pd
import torch from torchvision 
import transformsfrom PIL 
import Image from tqdm 
import tqdm from sklearn.preprocessing 
import MinMaxScaler
import umap
import matplotlib.pyplot as plt
import seaborn as sns from transformers import CLIPProcessor, CLIPModel

_HERE = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()
_ROOT = _HERE.parent  # project root
DATA_DIR = _ROOT / "data" / "paintings"
CSV_PATH = _ROOT / "data" / "paintings.csv"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

df = pd.read_csv(CSV_PATH)
print(f"Loaded {len(df)} rows")

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(DEVICE)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    return clip_processor(images=image, return_tensors="pt")["pixel_values"]

embeddings = []
failed_files = []

for filename in tqdm(df["filename"], desc="Extracting CLIP embeddings"):
    try:
        image_path = DATA_DIR / filename
        pixel_values = preprocess_image(image_path).to(DEVICE)

        with torch.no_grad():
            embedding = clip_model.get_image_features(pixel_values)
            embedding = embedding.squeeze().cpu().numpy()
            embeddings.append(embedding)
    except Exception as e:
        failed_files.append((filename, str(e)))
        embeddings.append([0.0] * 512)  # Dummy vector if fail

df["embedding"] = embeddings

embedding_matrix = pd.DataFrame(embeddings)
reducer = umap.UMAP(random_state=42)
embedding_2d = reducer.fit_transform(embedding_matrix)

df["x"] = embedding_2d[:, 0]
df["y"] = embedding_2d[:, 1]

scaler = MinMaxScaler()
df["year_norm"] = scaler.fit_transform(df[["year"]])

plt.figure(figsize=(12, 8))
scatter = plt.scatter(df["x"], df["y"], c=df["year_norm"], cmap="plasma", s=10)
cbar = plt.colorbar(scatter)
cbar.set_label("Year (normalized)")
plt.title("CLIP Embedding of Picasso Paintings (UMAP Projection)")
plt.xlabel("UMAP-1")
plt.ylabel("UMAP-2")
plt.grid(True)
plt.tight_layout()
plt.show()
